<a href="https://colab.research.google.com/github/Fatma-Hamouda/2025-Quantum-Factorization-With-Quantum-Rings/blob/main/Iquhack%20challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install QuantumRingsLib

In [4]:
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
from QuantumRingsLib import JobStatus
import numpy as np
import math
import time
from fractions import Fraction

# Initialize Quantum Rings Provider and Backend
provider = QuantumRingsProvider(
    token='rings-200.DTXB2V3yy9HYltRqPFs8dppsAESVGfdL',
    name='fatma_h00197@cic-cairo.com'
)
backend = provider.get_backend("scarlet_quantum_rings")
shots = 1024

In [9]:
# Helper Functions
def compute_gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

def modular_exponentiation_circuit(qc, a, N, control, work):
    n = len(control)
    precomputed_powers = [pow(a, 2**i, N) for i in range(n)]
    for i in range(n):
        a_exp = precomputed_powers[i]
        for j in range(len(work)):
            if (a_exp >> j) & 1:
                qc.cx(control[i], work[j])
    return qc

def iqft_cct(qc, control_register, n):
    for qubit in reversed(range(n)):
        qc.h(control_register[qubit])
        for j in reversed(range(qubit)):
            angle = -math.pi / (2 ** (qubit - j))
            qc.cp(angle, control_register[j], control_register[qubit])
    return qc

def extract_period(counts, n_qubits, a, N):
    measurements = [int(state, 2) for state in counts.keys()]
    if not measurements:
        print("No measurements found.")
        return None
    max_state = max(counts, key=lambda k: counts[k])
    s = int(max_state, 2)
    phase = s / (2 ** n_qubits)
    print(f"Phase: {phase}")
    frac = Fraction(phase).limit_denominator(N)
    print(f"Fraction: {frac}")
    r = frac.denominator
    if r % 2 == 0 and pow(a, r // 2, N) != N - 1:
        return r
    print(f"Invalid period: r={r}")
    return None

# Shor's Algorithm Implementation
def shors_algorithm(N):
    if N <= 1:
        raise ValueError("N must be greater than 1.")
    if N % 2 == 0:
        print(f"N is even. Factors: 2 and {N // 2}")
        return (2, N // 2)

    n_qubits = int(np.ceil(np.log2(N)))
    control_qubits = 2 * n_qubits
    work_qubits = n_qubits

    for attempt in range(1):
        a = np.random.randint(2, N)
        print(f"Attempt {attempt+1}: a = {a}")
        gcd = compute_gcd(a, N)
        if gcd != 1 and gcd != N:  # Ensure it's a non-trivial factor
            other_factor = N // gcd  # Compute the second factor
            print(f"Factors found: {gcd}, {other_factor}")
            return gcd, other_factor

        control = QuantumRegister(control_qubits, 'control')
        work = QuantumRegister(work_qubits, 'work')
        c = ClassicalRegister(control_qubits, 'c')
        qc = QuantumCircuit(control, work, c)

        qc.h(control)
        modular_exponentiation_circuit(qc, a, N, control, work)
        iqft_cct(qc, control, control_qubits)
        qc.measure(control, c)

        job = backend.run(qc, shots=shots)
        start_time = time.time()
        timeout = 1000

        while job.status() not in [JobStatus.DONE, JobStatus.CANCELLED, JobStatus.ERROR]:
            if time.time() - start_time > timeout:
                print("Job timed out.")
                job.cancel()
                break
            time.sleep(1)

        if job.status() == JobStatus.DONE:
            counts = job.result().get_counts()
            print(f"Measurement results: {counts}")
            r = extract_period(counts, control_qubits, a, N)
            if r:
                print(f"Period found: r = {r}")
                x = pow(a, r // 2, N)
                if x != N - 1:
                    factor1 = compute_gcd(x - 1, N)
                    factor2 = compute_gcd(x + 1, N)
                    print(f"Candidate factors: {factor1}, {factor2}")
                    if factor1 * factor2 == N:
                        print(f"Factors found: {factor1} and {factor2}")
                        return (factor1, factor2)

    print("Failed to find factors after multiple attempts.")
    return None



In [ ]:
# Main Execution
if __name__ == "__main__":
    N = 143  # Example semiprime
    print(f"Factoring N = {N}")
    factors = shors_algorithm(N)
    if factors:
        print(f"Factors of {N}: {factors}")
    else:
        print(f"Failed to factor {N}")

Factoring N = 143
Attempt 1: a = 72
